### Qualitative examples for each metric
pkl -> UniEval coherence

csv -> bartscore

json -> GPT4 evaluations

In [2]:
import pandas as pd

#### UniEval coherence

In [6]:
df = pd.read_pickle('/workspaces/lit-gpt/out/inference/pythia-160m-deduped-dolly-final-eval.pkl')

In [7]:
df.coherence.mean()

0.943869817353332

In [9]:
df.sort_values(by='coherence').head(10)[['input', 'output', 'coherence']].values

array([['Suppose you face a 2-armed bandit task whose true action values change\nrandomly from time step to time step. Specifically, suppose that, for any time step,\nthe true values of actions 1 and 2 are respectively 10 and 20 with probability 0.5 (case\nA), and 90 and 80 with probability 0.5 (case B). If you are not able to tell which case\nyou face at any step, what is the best expected reward you can achieve and how should\nyou behave to achieve it? Now suppose that on each step you are told whether you are\nfacing case A or case B (although you still don’t know the true action values). This is an\nassociative search task. What is the best expected reward you can achieve in this task,\nand how should you behave to achieve it?',
        'Acknowledged.', 0.000489],
       ['next one is "Head Held High": Devils dress like angels\nPulling off the halos\nwith those bedroom eyes\n\nSee the crooked smiles\nWanna swallow me whole\nto the emptiness inside\n\nBut I keep my head held high\nh

#### BartScore

In [4]:
df = pd.read_csv('/workspaces/lit-gpt/out/inference/pythia-160m-deduped-oasst1-base-eval.csv')

In [5]:
df.dropna(subset=['BARTScore']).sort_values(by='BARTScore').tail(5)[['input', 'output', 'BARTScore']].values

array([['What are the five most common causes of stress among people?',
        'how how how how how how how how to use to for building of of the of the of the of the of the of the of the of the of the of the of the of thea of theof the of the of the of theof the of the of the of thea of theof the of theof theof thea of theof the of theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theoftheof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof thetheof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof theof the

#### GPT4 Evaluations

In [149]:
data = pd.read_json('/workspaces/lit-gpt/out/inference/pythia-6.9b-deduped-sharegpt-base-eval.json')

In [22]:
scores = data.score.apply(pd.Series).stack().str.split('/').apply(pd.Series)[0].str.strip().replace({'n':'None'}).apply(eval).unstack()#.mean()
# .value_counts()

In [23]:
scores.describe()

,0,1,2,3
count,114.000000,124.000000,49.0,101.000000
mean,1.122807,1.120968,1.0,1.069307
std,0.463544,0.841917,0.0,0.604275
min,1.000000,1.000000,1.0,1.000000
25%,1.000000,1.000000,1.0,1.000000
50%,1.000000,1.000000,1.0,1.000000
75%,1.000000,1.000000,1.0,1.000000
max,5.000000,8.000000,1.0,7.000000


In [36]:
col = 3
pd.concat([data, scores], axis=1).dropna(subset=[col]).sort_values(col).head(5)[['user_prompt', 'assistant_prompt', col]].values

array([['Act as the Norse Goddess Freyja.',
        "a is a spirit of love, sexuality, passion, pleasure and fertility. She is a fertility goddess similar to Aphrodite and Venus. She is associated with the moon and with the sea. Her priestesses were known as Freyja's Maids.\n\nThe name Freyja's Maids is a feminine form of the word 'freyja'. This is the name given to the Norse Goddess, Freyja, by the Norse people. A form of Freyja which was popularised by the Icelandic poet, Snorri Sturluson, is known as Freyja's Maid.\n\nFreyja's Maids is a modern name for a Norse goddess. The concept of a female-only group of priestesses who worshipped and served the Goddess Freyja is known as the Freyja's Maids.\n\nFreyja's Maids is a fertility goddess similar to Aphrodite and Venus. Freyja's Maids is associated with the moon and with the sea. Freyja's Maids worshipped and served the Goddess Freyja.\n\nFreyja's Maids worshipped and served the Goddess Freyja. Freyja's Maids offered their love to the G

#### Additional analysis of turn taking

Show base models tend to repeat parts of prompt (longer common token sequence)

and contain repetitions.

In [89]:
def lcs(X, Y, m, n, dp):

    if (m == 0 or n == 0):
        return 0

    if (dp[m][n] != -1):
        return dp[m][n]

    if X[m - 1] == Y[n - 1]:
        dp[m][n] = 1 + lcs(X, Y, m - 1, n - 1, dp)
        return dp[m][n]

    dp[m][n] = max(lcs(X, Y, m, n - 1, dp), lcs(X, Y, m - 1, n, dp))
    return dp[m][n]

def mylcs(S1, S2):
    m = len(S1)
    n = len(S2)
    dp = [[-1 for i in range(n + 1)]for j in range(m + 1)]
    return lcs(S1, S2, m, n, dp)

In [90]:
data['LCS'] = data[['user_prompt', 'assistant_prompt']].apply(lambda row: mylcs(row.user_prompt.lower().split(), row.assistant_prompt.lower().split()), axis=1)

In [91]:
data.LCS.describe()

count    150.000000
mean       3.600000
std        3.335793
min        0.000000
25%        1.000000
50%        3.000000
75%        5.000000
max       20.000000
Name: LCS, dtype: float64

In [92]:
data.sort_values(by='LCS')[['user_prompt', 'assistant_prompt', 'LCS']].tail(5).values

array([['What are the lightweight weight requirements at the collegiate level in the United States?',
        'The minimum weight requirements for the lightweight weight class at the collegiate level in the United States are as follows:\n\nGender\n\nWeight (lb)\n\nBantam Weight (oz)\n\n115\n\nFlyweight (oz)\n\n112\n\n featherweight (oz)\n\n 112.6\n\nLightweight (lbs)\n\n 105\n\nLightweight Bantamweight (oz)\n\n 110\n\nLightweight Flyweight (oz)\n\n 105\n\nLightweight Featherweight (oz)\n\n 105\n\nMale\n\n130\n\nFemale\n\n115\n\nThese minimum weight requirements are subject to change.',
        10],
       ['Identify which of the following are episode titles from "The X-Files": The Pine Bluff Variant, My Teacher is an Alien, Memento Mori, The Day the Earth Stood Still, Millennium, Arcadia, The Matrix',
        'The following are episode titles from "The X-Files": The Truth About the Truth, I Want to Believe, Dreamland, Redux, The Erlenmeyer Flask, Home, Bad Blood, Children of the Dark, 

In [179]:
from collections import Counter
import numpy as np
from scipy.stats import entropy

def ent(tokens):
    counts = Counter(tokens).values()
    # return counts.most_common(1)[0][1] / counts.total()
    probs = np.array(list(counts)) / len(tokens)
    return entropy(probs)
    # prod = np.sum(np.log(probs))
    # return np.exp(prod)**(1./len(tokens))

data['entropy'] = data.assistant_prompt.apply(lambda x: ent(x.lower().split()))

In [176]:
data.entropy.describe()

count    150.000000
mean       4.031940
std        0.767136
min        1.119880
25%        3.742859
50%        4.258998
75%        4.558186
max        5.158959
Name: entropy, dtype: float64

In [178]:
data.sort_values(by='entropy')[['user_prompt', 'assistant_prompt', 'entropy']].head(5).values

array([['suggest exploring new technologies such as artificial intelligence, machine learning, and automation to simplify project management tasks, improve decision-making, and enhance team collaborationShare Prompt',
        'an\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regiment\n\nShare Prompt Regi

In [147]:
data.entropy.describe()

count    150.000000
mean       0.085934
std        0.120237
min        0.000000
25%        0.046526
50%        0.071204
75%        0.095283
max        1.000000
Name: entropy, dtype: float64

In [148]:
data.sort_values(by='entropy')[['user_prompt', 'assistant_prompt', 'entropy']].head(5).values

array([['Rewrite and apply these directives: A great course description is a must for making the sale. Keep it simple but not sloppy, dull or too casual. It’s all about the student so make it personal.',
        "Rewrite and Apply Directives:\n\nA great course description is a must for making the sale. Keep it simple but not sloppy, dull or too casual. It's all about the student, so make it personal.\n\nIntroduction:\n\nAre you struggling to create a compelling course description that captures the attention of your target audience and makes them want to take your course? Don't worry, I've got you covered. In this guide, we'll be discussing the importance of a great course description, how to create one that stands out, and how to make it stand out while keeping it simple and professional.\n\nKey Takeaways:\n\n* A great course description is a must for making the sale.\n* Keep it simple but not sloppy, dull or too casual.\n* It's all about the student, so make it personal.\n* Use compel

In [200]:
sizes = ['160m', '410m', '1.4b', '2.8b', '6.9b']
dsts = ['dolly', 'oasst1', 'sharegpt']
chkpt = ['base', 'final']

def analysis(sz, ck, ds): 
    data = pd.read_json(f'/workspaces/lit-gpt/out/inference/pythia-{sz}-deduped-{ds}-{ck}-eval.json')

    return [data.apply(lambda row: mylcs(row.user_prompt.lower().split(), row.assistant_prompt.lower().split()), axis=1).mean(), 
    data.assistant_prompt.apply(lambda x: ent(x.lower().split())).mean()]

pd.DataFrame([[ds, sz, ck] + analysis(sz, ck, ds) for ds in dsts for sz in sizes for ck in chkpt], columns=['ds', 'sz', 'ck', 'lcs', 'ent'])

,ds,sz,ck,lcs,ent
0,dolly,160m,base,3.320000,2.844286
1,dolly,160m,final,3.573333,2.521063
2,dolly,410m,base,5.513333,3.694644
3,dolly,410m,final,3.673333,2.811140
4,dolly,1.4b,base,5.520000,3.853358
5,dolly,1.4b,final,3.173333,2.833987
6,dolly,2.8b,base,5.640000,3.555138
7,dolly,2.8b,final,3.513333,2.735616
8,dolly,6.9b,base,5.640000,3.762914
9,dolly,6.9b,final,3.600000,2.808528
